# 4. Timing analysis

We will produce a light curve of PKS 2155-304 in two energy bands, compute the ratio of the fluxes and see if there is any hint of spectral variability. Based on this we will build a model based on two distinct time components and try to characterize their spectral and temporal properties.


In [ ]:
import matplotlib.pyplot as plt

import numpy as np

from astropy import units as u
from astropy.coordinates import SkyCoord, Angle
from astropy.time import Time
from astropy.table import Table

from gammapy.data import DataStore
from gammapy.datasets import Datasets, SpectrumDataset, SpectrumDatasetOnOff
from gammapy.estimators import FluxPointsEstimator, LightCurveEstimator
from gammapy.makers import (
    DatasetsMaker,
    SafeMaskMaker,
    SpectrumDatasetMaker,
    ReflectedRegionsBackgroundMaker,
)
from gammapy.maps import MapAxis, RegionGeom, WcsGeom
from gammapy.modeling import Fit
from gammapy.modeling.models import (
    PowerLawSpectralModel,
    PointSpatialModel,
    SkyModel,
)
from gammapy.utils import pbar
pbar.SHOW_PROGRESS_BAR = True

from gammapy.visualization import plot_spectrum_datasets_off_regions

from regions import CircleSkyRegion

We first load the relevant data set:

In [ ]:
data_store = DataStore.from_dir("$GAMMAPY_DATA/hess-dl3-dr1")

We select a sub-sample of data acquired on PKS 2155-304:

In [ ]:
target = SkyCoord(329.71693826 * u.deg, -30.2255890 * u.deg, frame="icrs")
selection = dict(
    type="sky_circle",
    frame="icrs",
    lon=target.ra,
    lat=target.dec,
    radius="2 deg",
)
selected_obs_table = data_store.obs_table.select_observations(selection)

obs_ids = selected_obs_table["OBS_ID"]

observations = data_store.get_observations(obs_ids)

Let's create a time intervals, for later use, and filter the observations on it:

In [ ]:
t0 = Time("2006-07-29T20:30")
duration = 10 * u.min
n_time_bins = 35

short_observations = 

In [ ]:
len(short_observations)

# Data reduction

Let's perform a 1D analysis of the data.

In [ ]:
on_region_radius = Angle("0.11 deg")

on_region = CircleSkyRegion(center=target, radius=on_region_radius)
exclusion_region = 

In [ ]:
energy_axis =  MapAxis.from_energy_bounds(0.1, 30, nbin=5, per_decade=True, unit="TeV", name="energy")
energy_axis_true = MapAxis.from_energy_bounds(0.05, 50, nbin=10, per_decade=True, unit="TeV", name="energy_true")

geom = RegionGeom.create(region=on_region, axes=[energy_axis])

dataset_empty = 
dataset_maker = 

bkg_maker = 
safe_mask_maker = 

In [ ]:
datasets = Datasets()

for obs in pbar.progress_bar(short_observations):
    dataset = 

In [ ]:
dataset_stack = datasets.stack_reduce()

Let's find some energy at which to slice our data, to build two well-balanced sets into two energy bands:

# Fit overall spectrum

In [ ]:
spectral_model = PowerLawSpectralModel()

In [ ]:
source = 
dataset_stack.models = 

In [ ]:
%%time
fit_stack = Fit()
result_stack = 

In [ ]:
print(result_stack)
display(result_stack.models.to_parameters_table())

In [ ]:
print(f"Pivot energy of the PL model:{source.spectral_model.pivot_energy}")

# define energy close to pivot as reference energy
source.spectral_model.reference.quantity = np.round(source.spectral_model.pivot_energy,1)

In [ ]:
result_stack_pivot = fit_stack.run(datasets=[dataset_stack])
display(result_stack_pivot.models.to_parameters_table())

In [ ]:
for result, color in zip([result_stack, result_stack_pivot], ["black", "red"]):
    spec_model = result.models[0].spectral_model
    spec_model.plot_error(energy_bounds=[100*u.GeV, 10*u.TeV], sed_type="e2dnde", energy_power=1, facecolor=color)

### Checking fit quality

Is the fit good?

Let's look at the overall fit statistic and residuals.

If number of counts per bins is large enough. WStat can provide an indicator of goodness of fit as it is asymptotically distributed as a chi-square with nbins - n free parameters dof.  

In [ ]:
ndof = np.sum(dataset_stack.mask) - 2
red_wstat = result_stack_pivot.total_stat /ndof

print(f"Reduced WStat is {red_wstat} for {ndof} d.o.f.")

In [ ]:
plt.figure(figsize=(5,5))
_=dataset_stack.plot_fit()

### Improve the model with curved PL

Use a log-parabola model

In [ ]:
lpl_source = 

dataset_stack.models = 

lpl_result = fit_stack.run([dataset_stack])

display(lpl_result.models.to_parameters_table())

In [ ]:
ndof = np.sum(dataset_stack.mask) - len(lpl_result.models.parameters.free_parameters)
red_wstat = lpl_result.total_stat /ndof

print(f"Reduced WStat is {red_wstat} for {ndof} d.o.f.")

plt.figure(figsize=(5,5))
_=dataset_stack.plot_fit()

### Improve the model taking into account EBL attenuation

Let's load precomputed EBL tables.

In [ ]:
from gammapy.modeling.models import EBLAbsorptionNormSpectralModel

ebl = EBLAbsorptionNormSpectralModel.read_builtin("dominguez")

ebl.redshift.value = 0.116

In [ ]:
absorbed_lpl = 

abs_lpl_source = 

dataset_stack.models = 

ebl_result = fit_stack.run([dataset_stack])
display(ebl_result.models.to_parameters_table())

In [ ]:
ndof = np.sum(dataset_stack.mask) - len(ebl_result.models.parameters.free_parameters)
red_wstat = ebl_result.total_stat /ndof

print(f"Reduced WStat is {red_wstat} for {ndof} d.o.f.")

plt.figure(figsize=(5,5))
_ = dataset_stack.plot_fit()

In [ ]:
%%time

# Compute flux points
from gammapy.estimators.utils import resample_energy_edges
energy_edges = resample_energy_edges(dataset_stack, conditions={'counts_min': 5.})
print(energy_edges)

fpe = 

Let's plot the SED so far:

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6)) 

# plot joint model
abs_lpl_source.spectral_model.plot( energy_bounds=[200*u.GeV, 20*u.TeV], sed_type="e2dnde")
abs_lpl_source.spectral_model.plot_error(energy_bounds=[200*u.GeV, 20*u.TeV], sed_type="e2dnde", facecolor="grey", alpha=0.7)

ax = flux_points.plot(ax=ax,sed_type="e2dnde")
ax.set_xlim(0.2,30)
ax.set_ylim(1e-14, 1e-9)

# Light curve

Let's compute the light curve in two energy ranges

In [ ]:
datasets.models = [abs_lpl_source]

In [ ]:
lc_maker = 
lc = lc_maker.run(datasets)

Plot the light curves:

In [ ]:
lc.plot(sed_type="flux", time_format="mjd", axis_name="time")

# Fractional and point-to-point variability

In [ ]:
# Compute the global fractional variability, for each energy intervals

In [ ]:
# Compute the point-to-point fractional variability, for each energy intervals

In [ ]:
# Compute the characteristic doubling time of the light curves, for each energy intervals

# Hardness ratio diagrams

Let's compute the flux ratio of our two light curves, plot them against time, and against the overall flux (i.e. hardness ratio diagram).

Access the low-energy and high-energy light curves.

**Tip**: Remember that `RegionNDMap` holds quantities of `numpy.ndarray`s.

In [ ]:
lc_lo = lc.flux.quantity[:,0,...].squeeze()
lc_hi = lc.flux.quantity[:,1,...].squeeze()

lc_lo_err = lc.flux_err.quantity[:,0,...].squeeze()
lc_hi_err = lc.flux_err.quantity[:,1,...].squeeze()

In [ ]:
time = lc.geom.axes["time"].center - lc.geom.axes["time"].center[0]
plt.errorbar(x=lc_lo, y=lc_hi, xerr=lc_lo_err, yerr=lc_hi_err, linestyle='', linewidth=0.5)
plt.scatter(lc_lo, lc_hi, c=time.to_value("h"))

plt.xlabel(r'Low energy flux $[\rm cm^{-2}\ s^{-1}]$')
plt.ylabel(r"High energy flux $[\rm cm^{-2}\ s^{-1}]$")
plt.colorbar().set_label("time since start [h]")
plt.show()

In [ ]:
flux_ratio = lc_hi/lc_lo
flux_ratio_err = flux_ratio * np.sqrt((lc_hi_err/lc_hi)**2+(lc_lo_err/lc_lo)**2)

In [ ]:
# Plot the flux ratio versus time

plt.errorbar(x=lc.geom.axes["time"].time_mid.mjd, y=np.squeeze(flux_ratio), yerr=np.squeeze(flux_ratio_err), fmt='o')
plt.xlabel("Time (MJD)")
plt.ylabel(f"Hardness ratio")

Now, let's plot the hardness ratio diagram (integral flux in the whole energy range, versus the flux ratio)

In [ ]:
def time_resolved_spectroscopy(datasets, model, time_intervals):
    fit = Fit()
    valid_intervals = []
    fit_results = []
    index = 0
    for t_min, t_max in time_intervals:
        datasets_to_fit = datasets.select_time(time_min=t_min, time_max=t_max)

        if len(datasets_to_fit) == 0:
            print(
                f"No Dataset for the time interval {t_min} to {t_max}. Skipping interval."
            )
            continue

        model_in_bin = model.copy(name="Model_bin_" + str(index))
        datasets_to_fit.models = model_in_bin
        result = fit.run(datasets_to_fit)
        fit_results.append(result)
        valid_intervals.append([t_min, t_max])
        index += 1

    return valid_intervals, fit_results

In [ ]:
abs_lpl_source.parameters["beta"].frozen=True
valid_times, results = time_resolved_spectroscopy(datasets, abs_lpl_source, time_intervals)

In [ ]:
data = []
for valid_time, fit_result in zip(valid_times, results):
    result_dict = {}
    result_dict["tstart"] = valid_time[0]
    result_dict["tstop"] = valid_time[1]
    
    for par in fit_result.models[0].parameters.free_parameters:
        result_dict[par.name] = par.quantity
        result_dict[f"{par.name} error"] = par.error * par.unit

    result_dict["total_stat"] = fit_result.total_stat
    result_dict["success"] = fit_result.success

    data.append(result_dict)

table = Table(data)

In [ ]:
table = table[table["success"]==True]
from gammapy.maps import TimeMapAxis
time_axis = TimeMapAxis.from_time_edges(
    time_min=table["tstart"], time_max=table["tstop"]
)

fix, axes = plt.subplots(2, 1, figsize=(8, 8))
axes[0].errorbar(
    x=time_axis.as_plot_center, y=table["alpha"], yerr=table["alpha error"], fmt="o"
)

axes[1].errorbar(
    x=time_axis.as_plot_center,
    y=table["amplitude"],
    yerr=table["amplitude error"],
    fmt="o",
)

axes[0].set_ylabel("alpha")
axes[1].set_ylabel("amplitude")
axes[1].set_xlabel("time")
plt.show()

In [ ]:
plt.errorbar(
    table["amplitude"],
    table["alpha"],
    xerr=table["amplitude error"],
    yerr=table["alpha error"],
    linestyle="",
    linewidth=0.5,
)
plt.scatter(table["amplitude"], table["alpha"], c=time_axis.center.value)
plt.xlabel(r"amplitude $\rm [cm^{-2}\,s^{-1}]$")
plt.ylabel("alpha")
plt.xscale("log")
plt.colorbar().set_label("time [h]")
plt.show()

## Are there different overlapping time components?

The time resolved spectroscopy supports the idea that PKS 2155-304 spectrum is harder and brighter in the first part of the night. We can distinguish two populations of points in the time resolved spectral parameters. We try to explore this idea in the following trying to fit different time and spectral components to the data.

Here we will use full sky models with time and spectral models. We will first try to see whether two separate components are required to fit the data. Then we will determine the best spectral representations for these time components.

First we introduce some convenience plotting functions to compare a model with a lightcurve.

In [ ]:
def plot_compare_lc_model(ebin, lc, cst, flare, colors=["blue", "red"]):
    emin, emax = lc.geom.axes["energy"].edges_min[ebin], lc.geom.axes["energy"].edges_max[ebin]

    ax = lc.slice_by_energy(emin, emax).plot(sed_type="flux")
    ax.set_yscale("linear")
    time_values = lc.geom.axes["time"].time_mid
    xt =lc.geom.axes["time"].as_plot_center

    values = []
    for model, color in zip([cst, flare], colors):
        flux_values = model.temporal_model(time_values)*model.spectral_model.integral(emin, emax)
        ax.plot(xt, flux_values, label=model.name, color=color)
        values.append(flux_values)
        
    ax.plot(xt, values[0]+values[1], label="total", color='k')
    
    return ax

def plot_compare_spec_models(cst, flare, colors=["blue", "red"]):
    for model, color in zip([cst, flare], colors):
        ax=model.spectral_model.plot(energy_bounds=[0.2, 10]*u.TeV, sed_type="e2dnde", color=color, label=model.name)
        ax=model.spectral_model.plot_error(energy_bounds=[0.2, 10]*u.TeV, sed_type="e2dnde")
    ax.legend()

def summary_plot(lc, cst, flare):
    plt.figure(figsize=(14,4))
    ax1 = plt.subplot(131)
    plot_compare_lc_model(0, lc, cst, flare)
    ax2 = plt.subplot(132)
    plot_compare_lc_model(1, lc, cst, flare)
    ax3 = plt.subplot(133)
    plot_compare_spec_models(cst, flare)

### Are there two temporal components?

We first define two temproal models:
- a steady component modeled with a `ConstantTemporalModel`
- a flare component modeled with a `GeneralizedGaussianTemporalModel`. The latter allows for independent rise    and decay times. We set the peak time following the LC and for now freeze the parameter controling the shape   of the model.

Gammapy offers a convenience function to test nested models: `select_nested_models`.

In [ ]:
from gammapy.modeling.models import GeneralizedGaussianTemporalModel, ConstantTemporalModel
from gammapy.modeling import select_nested_models

In [ ]:
flare_time_model = GeneralizedGaussianTemporalModel(t_ref="53945.9 d",t_rise="0.3 h", t_decay="2 h", eta=0.5)
flare_time_model.eta.frozen=True
flare_time_model.t_rise.min=0.01
flare_time_model.t_rise.max=3
flare_time_model.t_decay.min=0.1
flare_time_model.t_decay.max=100
flare_time_model.t_ref.frozen=True

cst_time_model = ConstantTemporalModel()

In [ ]:
spectral_model = abs_lpl_source.spectral_model.copy()
spectral_model.parameters["alpha"].frozen=True
spectral_model.parameters["beta"].frozen=True

cst = 
flare = 

In [ ]:
datasets.models = 

In [ ]:
result_two_flare = select_nested_models(
    datasets, 
    parameters=[cst.parameters["amplitude"]], 
    null_values=[0.]
)

In [ ]:
print(f"Fit improvement : Delta TS = {result_two_flare['ts']}")
summary_plot(lc, cst, flare)
#display(result_two_flare["fit_results"].models.to_parameters_table())

There is a very significant preference for a steady model in addition to the flare.

Let's now try to see if the spectral parameters of the flare are different from the average spectrum

In [ ]:
flare.parameters["alpha"].frozen = False
flare.parameters["beta"].frozen = False

result_two_flare = select_nested_models(
    datasets, 
    parameters=[flare.parameters["alpha"], flare.parameters["beta"]], 
    null_values=[flare.parameters["alpha"].value, flare.parameters["beta"].value]
)

print(f"Fit improvement : Delta TS = {result_two_flare['ts']}")
#display(result_two_flare["fit_results"].models.to_parameters_table())
summary_plot(lc, cst, flare)

The flare spectrum is significantly different from the average one. What about the steady component?


In [ ]:
cst.parameters["alpha"].frozen = False
cst.parameters["beta"].frozen = False

result_two_flare = select_nested_models(
    datasets, 
    parameters=[cst.parameters["alpha"], cst.parameters["beta"]], 
    null_values=[cst.parameters["alpha"].value, cst.parameters["beta"].value]
)

print(f"Fit improvement : Delta TS = {result_two_flare['ts']}")
summary_plot(lc, cst, flare)

The comparison of the LC with the fitted model shows that the flare structure seems to be more peaked. Let's release the remaining parameters of the generalized gaussian (peak position and shape parameter).

In [ ]:
flare.parameters["eta"].frozen = False
flare.parameters["eta"].min = 0.1
flare.parameters["eta"].max = 3

flare.parameters["t_ref"].frozen = False
flare.parameters["t_ref"].min = flare.parameters["t_ref"].value - 0.3
flare.parameters["t_ref"].max = flare.parameters["t_ref"].value + 0.3

result_two_flare = select_nested_models(
    datasets, 
    parameters=[flare.parameters["eta"],flare.parameters["t_ref"]], 
    null_values=[flare.parameters["eta"].value,flare.parameters["t_ref"].value]
)

print(f"Fit improvement : Delta TS = {result_two_flare['ts']}")
summary_plot(lc, cst, flare)

The final model parameters are given below:

In [ ]:
result_two_flare["fit_results"].models.to_parameters_table()